In [1]:
from IPython import __version__ as ipython_version
from pandas import __version__ as pandas_version
from bokeh import __version__ as bokeh_version
print(f'IPython - {ipython_version}')
print(f'Pandas - {pandas_version}')
print(f'Bokeh - {bokeh_version}')

IPython - 6.1.0
Pandas - 0.22.0
Bokeh - 0.12.14


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, date

<H2>Stage Ranking codes</H2>

In [3]:
# read VWSTAGERANKING.csv and drop unused fields
stgrnk = pd.read_csv('VWSTAGERANKING.csv')
stgrnk.drop(['code_table', 'MEDIUM_DESC',
             'Converted/Confirmed/Accepted/Require SepDate'],
            inplace=True, axis=1)

In [ ]:
print('stgrnk', stgrnk.shape)

In [ ]:
print('stgrnk\n', stgrnk.dtypes)

In [ ]:
print(stgrnk.columns)

In [78]:
# stgrnk[stgrnk['status']=='A'].to_csv('stgrnk.csv')

<H2>Stage History data</H2>

In [4]:
# read STAGEHISTORY.csv
stg_hist_dtype = {'PEOPLE_CODE_ID': str, 'ACADEMIC_YEAR': str,
                  'ACADEMIC_TERM': str, 'ACADEMIC_SESSION': str,
                  'FIELD_ID': np.int64}
date_cols = ['FIELD_DATE']
stg_hist = pd.read_csv('STAGEHISTORY.csv', dtype=stg_hist_dtype,
                       parse_dates=date_cols,
                       usecols=['PEOPLE_CODE_ID', 'ACADEMIC_YEAR',
                                'ACADEMIC_TERM', 'ACADEMIC_SESSION',
                                'FIELD_ID', 'FIELD_DATE'])

In [79]:
print('stg_hist', stg_hist.shape)

stg_hist (279835, 7)


In [ ]:
print('stg_hist')
print(stg_hist.dtypes)

In [5]:
stg_hist['create_date'] = stg_hist['FIELD_DATE']

In [6]:
stage_data = pd.merge(stg_hist, stgrnk, left_on=['FIELD_ID'],
                      right_on=['STAGERANKING_ID'], how='left')

In [7]:
keep_fields = ['PEOPLE_CODE_ID', 'ACADEMIC_YEAR', 'ACADEMIC_TERM',
               'ACADEMIC_SESSION', 'field_name', 'field_value', 'create_date']
stage_data = stage_data.loc[~stage_data['create_date'].isnull(), keep_fields]

In [80]:
print('stage_data', stage_data.shape)

stage_data (279835, 7)


In [ ]:
print('stage_data')
print(stage_data.dtypes)
stage_data.head()

<H2>Academic Data</H2>

In [8]:
academic_dtype = {'PEOPLE_CODE_ID': str, 'ACADEMIC_YEAR': str,
                  'ACADEMIC_TERM': str, 'ACADEMIC_SESSION': str,
                  'APPLICATION_FLAG': str, 'APP_STATUS': str}
date_cols = ['APPLICATION_DATE', 'APP_STATUS_DATE', 'APP_DECISION_DATE']
academic = pd.read_csv('ACADEMIC.csv', dtype=academic_dtype,
                       parse_dates=date_cols,
                       usecols=['PEOPLE_CODE_ID',
                                'ACADEMIC_YEAR', 'ACADEMIC_TERM',
                                'ACADEMIC_SESSION', 'POPULATION',
                                'INQUIRY_FLAG',
                                'APPLICATION_FLAG', 'APPLICATION_DATE',
                                'APP_STATUS', 'APP_STATUS_DATE',
                                'APP_DECISION', 'APP_DECISION_DATE'])

In [ ]:
print('academic', academic.shape)
print('academic')
print(academic.dtypes)
academic.info()

In [ ]:
print(academic['POPULATION'].value_counts().sort_index())
print(academic['POPULATION'].value_counts().sum())

In [9]:
app_data = (academic.loc[~(academic['POPULATION'].isin(['ADVSTU', 'NOND'])) &
                         ((academic['INQUIRY_FLAG'] == 'Y') |
                          (academic['APPLICATION_FLAG'] == 'Y'))]
            )

print('app_data', app_data.shape)
print('app_data')
print(app_data.dtypes)

app_data (42151, 12)
app_data
PEOPLE_CODE_ID               object
ACADEMIC_YEAR                object
ACADEMIC_TERM                object
ACADEMIC_SESSION             object
POPULATION                   object
APPLICATION_FLAG             object
APP_STATUS                   object
APP_STATUS_DATE      datetime64[ns]
APP_DECISION                 object
APP_DECISION_DATE    datetime64[ns]
INQUIRY_FLAG                 object
APPLICATION_DATE     datetime64[ns]
dtype: object


In [ ]:
app_data.info()

In [10]:
applied = (app_data[app_data['APP_STATUS'].notnull()]
           .rename(columns={'APP_STATUS': 'field_value'})
           .rename(columns={'APP_STATUS_DATE': 'create_date'})
           )
applied.loc[:, 'field_name'] = 'Application Status'
applied = applied.loc[~applied['create_date'].isnull(), keep_fields]

print('applied', applied.shape)
print('applied')
print(applied.dtypes)

applied (14387, 7)
applied
PEOPLE_CODE_ID              object
ACADEMIC_YEAR               object
ACADEMIC_TERM               object
ACADEMIC_SESSION            object
field_name                  object
field_value                 object
create_date         datetime64[ns]
dtype: object


In [81]:
print(applied['field_value'].value_counts().sort_index())
print(applied['field_value'].value_counts().sum())

300     1516
500     8280
ACXL    1705
CANC    2546
COMP     152
DEFR     185
nosh       3
Name: field_value, dtype: int64
14387


In [ ]:
applied.info()

In [11]:
accepted = (app_data[app_data['APP_DECISION'].notnull()]
            .rename(columns={'APP_DECISION': 'field_value'})
            .rename(columns={'APP_DECISION_DATE': 'create_date'})
            )
accepted.loc[:, 'field_name'] = 'Application Decision'
accepted = accepted.loc[~accepted['create_date'].isnull(), keep_fields]


print('accepted', accepted.shape)
print('accepted')
print(accepted.dtypes)

accepted (14290, 7)
accepted
PEOPLE_CODE_ID              object
ACADEMIC_YEAR               object
ACADEMIC_TERM               object
ACADEMIC_SESSION            object
field_name                  object
field_value                 object
create_date         datetime64[ns]
dtype: object


In [82]:
print(accepted['field_value'].value_counts().sort_index())
print(accepted['field_value'].value_counts().sum())

ACC     7555
CANC     165
DEF       10
DENY      39
DPAC    3903
PEND    1845
TRDP     498
TRNS     272
nosh       3
Name: field_value, dtype: int64
14290


In [ ]:
accepted.info()

In [12]:
# stack Stage History, Academic Applied and Academic Accepted
adm_df = stage_data.append(applied).append(accepted)

print('adm_df', adm_df.shape)
print('adm_df')
print(adm_df.dtypes)

adm_df (308512, 7)
adm_df
PEOPLE_CODE_ID              object
ACADEMIC_YEAR               object
ACADEMIC_TERM               object
ACADEMIC_SESSION            object
field_name                  object
field_value                 object
create_date         datetime64[ns]
dtype: object


In [ ]:
adm_df.head()

In [ ]:
adm_df.info()

In [13]:
adm_df = (adm_df.loc[((adm_df['ACADEMIC_TERM'].isin(['FALL', 'SPRING'])) &
                      (adm_df['ACADEMIC_SESSION'] == 'MAIN') &
                      (adm_df['ACADEMIC_YEAR'] >= '2009')
                      )
                     ]
          )

print('adm_df', adm_df.shape)
print('adm_df')
print(adm_df.dtypes)

adm_df (118325, 7)
adm_df
PEOPLE_CODE_ID              object
ACADEMIC_YEAR               object
ACADEMIC_TERM               object
ACADEMIC_SESSION            object
field_name                  object
field_value                 object
create_date         datetime64[ns]
dtype: object


In [14]:
# create new fields
adm_df['year_term'] = (adm_df['ACADEMIC_YEAR'] + '.' +
                       adm_df['ACADEMIC_TERM'].str.title())
adm_df['Week_Number'] = adm_df['create_date'].dt.week

In [15]:
# convert ACADEMIC_YEAR to numeric keep numeric-valued records
adm_df['ACADEMIC_YEAR'] = pd.to_numeric(adm_df['ACADEMIC_YEAR'],
                                        errors='coerce', downcast='integer')
adm_df = adm_df.loc[adm_df['ACADEMIC_YEAR'].notnull()]

In [16]:
adm_week_number = (lambda r: (r['Week_Number'] -
                              (date(int(r['ACADEMIC_YEAR']), 9, 1)
                              .isocalendar()[1])
                              )
                   if (r['Week_Number'] > (date(int(r['ACADEMIC_YEAR']), 9, 1)
                                           .isocalendar()[1]))
                   else (53 + r['Week_Number'] -
                         (date(int(r['ACADEMIC_YEAR']), 9, 1)
                         .isocalendar()[1])
                         )
                   )


In [17]:
adm_df['Admissions_Week'] = adm_df.apply(adm_week_number, axis=1)

print('adm_df', adm_df.shape)
print('adm_df')
print(adm_df.dtypes)

adm_df (118325, 10)
adm_df
PEOPLE_CODE_ID              object
ACADEMIC_YEAR                int16
ACADEMIC_TERM               object
ACADEMIC_SESSION            object
field_name                  object
field_value                 object
create_date         datetime64[ns]
year_term                   object
Week_Number                  int64
Admissions_Week              int64
dtype: object


In [ ]:
adm_df.info()

In [ ]:
adm_df.head()

In [ ]:
adm_df.columns

In [ ]:
adm_df[(adm_df['year_term']=='2014.Fall') & (adm_df['PEOPLE_CODE_ID']=='P000026232')].head(30)

In [ ]:
print(adm_df['field_value'].value_counts().sort_index())
print(adm_df['field_value'].value_counts().sum())

In [18]:
adm_keep_values = ['300', 'ACC', 'ACXL', 'CANC', 'DEF', 'DEFR', 'DENY', 'DPAC',
                   'TRDP', 'TRPD', 'TRNS', 'WAIT']
adm_keep_cols = ['PEOPLE_CODE_ID', 'year_term', 'Admissions_Week',
                 'field_value']
adm_df = adm_df.loc[(adm_df['field_value'].isin(adm_keep_values)),
                    adm_keep_cols]

print('adm_df', adm_df.shape)
print('adm_df')
print(adm_df.dtypes)

adm_df (36706, 4)
adm_df
PEOPLE_CODE_ID     object
year_term          object
Admissions_Week     int64
field_value        object
dtype: object


In [ ]:
adm_df.head()

In [ ]:
adm_df[(adm_df['year_term']=='2014.Fall') & (adm_df['PEOPLE_CODE_ID']=='P000026232')].head(30)

In [19]:
# admissions status table
admission_status = {'300': 'Applied', 'ACC': 'Accepted', 'ACXL': 'Canceled',
                    'CANC': 'Canceled', 'DEF': 'Canceled', 'DEFR': 'Canceled',
                    'DENY': 'Canceled', 'DPAC': 'Deposited',
                    'TRDP': 'Deposited', 'TRPD': 'Deposited',
                    'TRNS': 'Accepted', 'WAIT': 'Accepted'}
adm_stat = pd.DataFrame(list(admission_status.items()),
                        columns=['field_value', 'admission_status'])

In [20]:
adm_df1 = (pd.merge(adm_df, adm_stat, on=['field_value'], how='left')
           .drop(['field_value'], axis=1)
           .drop_duplicates(['PEOPLE_CODE_ID', 'year_term', 'Admissions_Week',
                             'admission_status'])
           )

print('adm_df1', adm_df1.shape)
print('adm_df1')
print(adm_df1.dtypes)

adm_df1 (22992, 4)
adm_df1
PEOPLE_CODE_ID      object
year_term           object
Admissions_Week      int64
admission_status    object
dtype: object


In [ ]:
adm_df1.head()

In [ ]:
adm_df1[(adm_df1['year_term']=='2014.Fall') & (adm_df1['admission_status']=='Deposited')].head(30)

In [ ]:
adm_df1[(adm_df1['year_term']=='2014.Fall') & (adm_df1['PEOPLE_CODE_ID']=='P000026232')].head(30)

In [21]:
adm_df1 = (adm_df1.sort_values(['year_term', 'PEOPLE_CODE_ID',
                                'Admissions_Week'])
           .drop_duplicates(['year_term', 'PEOPLE_CODE_ID',
                             'admission_status'],
                            keep='first')
           )

In [ ]:
adm_df1[(adm_df1['year_term']=='2014.Fall') & (adm_df1['admission_status']=='Deposited')].head(30)

In [ ]:
adm_df1[(adm_df1['year_term']=='2014.Fall') & (adm_df1['PEOPLE_CODE_ID']=='P000026232')].head(30)

In [ ]:
adm_df2 = (adm_df1.loc[(adm_df1['year_term'].isin(['2014.Fall', '2015.Fall']))]
           .reset_index()
           .set_index(['year_term', 'PEOPLE_CODE_ID', 'admission_status'])
           .drop(['index'], axis=1)
           .unstack(level=-1)
          )

print('adm_df2', adm_df2.shape)
print('adm_df2')
print(adm_df2.dtypes)

In [ ]:
adm_df2.head()

In [ ]:
#q = adm_df1.loc[(adm_df1['year_term']=='2014.Fall'), ['PEOPLE_CODE_ID', 'Admissions_Week', 'admission_status']]
q = adm_df1.loc[((adm_df1['year_term']=='2014.Fall') | (adm_df1['year_term']=='2015.Fall')), ['year_term', 'PEOPLE_CODE_ID', 'Admissions_Week', 'admission_status']]

In [ ]:
print(q.shape)
q.head()

In [ ]:
w = q.set_index(['year_term', 'PEOPLE_CODE_ID'])

In [ ]:
print(w.shape)
w.head(30)

In [22]:
e = adm_df1.pivot_table(index=['year_term', 'PEOPLE_CODE_ID'],
                        columns=['admission_status'],
                        values=['Admissions_Week']
                        )
print(e.shape)
e.head(30)

(10729, 4)


Admissions_Week                           
admission_status                Accepted Applied Canceled Deposited
year_term PEOPLE_CODE_ID                                           
2009.Fall P000000564                25.0     NaN      NaN       NaN
          P000006539                37.0     NaN      NaN       NaN
          P000022053                29.0     NaN      NaN       NaN
          P000022130                28.0     NaN      NaN       NaN
          P000022259                26.0     NaN      NaN       NaN
          P000022701                50.0    50.0      NaN       NaN
          P000022722                 3.0     3.0      NaN       NaN
          P000022725                 NaN    49.0     49.0       NaN
          P000022727                 5.0     NaN      NaN       NaN
          P000022729                 6.0     NaN      NaN       NaN
          P000022730                 6.0     NaN      NaN       NaN
          P000022734                 7.0     NaN      NaN       NaN
          P000022736                 7.0     NaN      NaN       NaN
          P000022758                 9.0     NaN      NaN       NaN
          P000022759                 9.0     NaN      NaN       NaN
          P000022761                 9.0     NaN      NaN       NaN
          P000022762                 9.0     NaN      NaN       NaN
          P000022768                10.0     NaN      NaN       NaN
          P000022769                10.0     NaN      NaN       NaN
          P000022774                11.0     NaN     41.0       NaN
          P000022776                11.0     NaN      NaN       NaN
          P000022782                11.0     NaN      NaN       NaN
          P000022783                11.0     NaN      NaN       NaN
          P000022785                11.0     NaN      NaN       NaN
          P000022787                12.0     NaN      NaN       NaN
          P000022788                12.0     NaN      NaN       NaN
          P000022796                13.0     NaN      NaN       NaN
          P000022797                13.0     NaN      NaN       NaN
          P000022798                13.0     NaN      NaN       NaN
          P000022799                13.0     NaN      NaN       NaN

In [23]:
# function returns status for week
def f_status(field, data_frame, n):
    f_week = (lambda df: 1
              if ((df[('Admissions_Week', field)] <= n) &
                  (df[('Admissions_Week', 'Canceled')] > n))
              else 0
              )
    return data_frame.apply(f_week, axis=1)

In [48]:
# function returns DataFrame of 53 week status values
def fill_weeks(field, data_frame):
    weeks = range(1, 54)
    r = pd.DataFrame(np.zeros((data_frame.shape[0], 53)),
                     index=data_frame.index,
                     columns=[f'{w:02d}' for w in weeks])
    for w in weeks:
        f = f'{w:02d}'
        r.loc[:, f] = f_status(field, data_frame, w)
        r.loc[:, 'stage'] = field

    r = r.reset_index().set_index(['year_term', 'stage', 'PEOPLE_CODE_ID'])
   
    return r

In [49]:
stage_list = ['Applied', 'Accepted', 'Deposited']
p = pd.DataFrame()
for stg in stage_list:
    p = pd.concat([p, fill_weeks(stg, e)])


In [50]:
p.head()


01  02  03  04  05  06  07  08  09  10 ...  \
year_term stage   PEOPLE_CODE_ID                                         ...   
2009.Fall Applied P000000564       0   0   0   0   0   0   0   0   0   0 ...   
                  P000006539       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022053       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022130       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022259       0   0   0   0   0   0   0   0   0   0 ...   

                                  44  45  46  47  48  49  50  51  52  53  
year_term stage   PEOPLE_CODE_ID                                          
2009.Fall Applied P000000564       0   0   0   0   0   0   0   0   0   0  
                  P000006539       0   0   0   0   0   0   0   0   0   0  
                  P000022053       0   0   0   0   0   0   0   0   0   0  
                  P000022130       0   0   0   0   0   0   0   0   0   0  
                  P000022259       0   0   0   0   0   0   0   0   0   0  

[5 rows x 53 columns]

In [51]:

print(p.shape)
p.head(30)

(32187, 53)


01  02  03  04  05  06  07  08  09  10 ...  \
year_term stage   PEOPLE_CODE_ID                                         ...   
2009.Fall Applied P000000564       0   0   0   0   0   0   0   0   0   0 ...   
                  P000006539       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022053       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022130       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022259       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022701       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022722       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022725       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022727       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022729       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022730       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022734       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022736       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022758       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022759       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022761       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022762       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022768       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022769       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022774       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022776       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022782       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022783       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022785       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022787       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022788       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022796       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022797       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022798       0   0   0   0   0   0   0   0   0   0 ...   
                  P000022799       0   0   0   0   0   0   0   0   0   0 ...   

                                  44  45  46  47  48  49  50  51  52  53  
year_term stage   PEOPLE_CODE_ID                                          
2009.Fall Applied P000000564       0   0   0   0   0   0   0   0   0   0  
                  P000006539       0   0   0   0   0   0   0   0   0   0  
                  P000022053       0   0   0   0   0   0   0   0   0   0  
                  P000022130       0   0   0   0   0   0   0   0   0   0  
                  P000022259       0   0   0   0   0   0   0   0   0   0  
                  P000022701       0   0   0   0   0   0   0   0   0   0  
                  P000022722       0   0   0   0   0   0   0   0   0   0  
                  P000022725       0   0   0   0   0   0   0   0   0   0  
                  P000022727       0   0   0   0   0   0   0   0   0   0  
                  P000022729       0   0   0   0   0   0   0   0   0   0  
                  P000022730       0   0   0   0   0   0   0   0   0   0  
                  P000022734       0   0   0   0   0   0   0   0   0   0  
                  P000022736       0   0   0   0   0   0   0   0   0   0  
                  P000022758       0   0   0   0   0   0   0   0   0   0  
                  P000022759       0   0   0   0   0   0   0   0   0   0  
                  P000022761       0   0   0   0   0   0   0   0   0   0  
                  P000022762       0   0   0   0   0   0   0   0   0   0  
                  P000022768       0   0   0   0

In [52]:
p

01  02  03  04  05  06  07  08  09  10  \
year_term   stage     PEOPLE_CODE_ID                                           
2009.Fall   Applied   P000000564       0   0   0   0   0   0   0   0   0   0   
                      P000006539       0   0   0   0   0   0   0   0   0   0   
                      P000022053       0   0   0   0   0   0   0   0   0   0   
                      P000022130       0   0   0   0   0   0   0   0   0   0   
                      P000022259       0   0   0   0   0   0   0   0   0   0   
                      P000022701       0   0   0   0   0   0   0   0   0   0   
                      P000022722       0   0   0   0   0   0   0   0   0   0   
                      P000022725       0   0   0   0   0   0   0   0   0   0   
                      P000022727       0   0   0   0   0   0   0   0   0   0   
                      P000022729       0   0   0   0   0   0   0   0   0   0   
                      P000022730       0   0   0   0   0   0   0   0   0   0   
                      P000022734       0   0   0   0   0   0   0   0   0   0   
                      P000022736       0   0   0   0   0   0   0   0   0   0   
                      P000022758       0   0   0   0   0   0   0   0   0   0   
                      P000022759       0   0   0   0   0   0   0   0   0   0   
                      P000022761       0   0   0   0   0   0   0   0   0   0   
                      P000022762       0   0   0   0   0   0   0   0   0   0   
                      P000022768       0   0   0   0   0   0   0   0   0   0   
                      P000022769       0   0   0   0   0   0   0   0   0   0   
                      P000022774       0   0   0   0   0   0   0   0   0   0   
                      P000022776       0   0   0   0   0   0   0   0   0   0   
                      P000022782       0   0   0   0   0   0   0   0   0   0   
                      P000022783       0   0   0   0   0   0   0   0   0   0   
                      P000022785       0   0   0   0   0   0   0   0   0   0   
                      P000022787       0   0   0   0   0   0   0   0   0   0   
                      P000022788       0   0   0   0   0   0   0   0   0   0   
                      P000022796       0   0   0   0   0   0   0   0   0   0   
                      P000022797       0   0   0   0   0   0   0   0   0   0   
                      P000022798       0   0   0   0   0   0   0   0   0   0   
                      P000022799       0   0   0   0   0   0   0   0   0   0   
...                                   ..  ..  ..  ..  ..  ..  ..  ..  ..  ..   
2018.Fall   Deposited P000054631       0   0   0   0   0   0   0   0   0   0   
                      P000054632       0   0   0   0   0   0   0   0   0   0   
                      P000054633       0   0   0   0   0   0   0   0   0   0   
                      P000054634       0   0   0   0   0   0   0   0   0   0   
                      P000054635       0   0   0   0   0   0   0   0   0   0   
                      P000054636       0   0   0   0   0   0   0   0   0   0   
                      P000054641       0   0   0   0   0   0   0   0   0   0   
                      P000054642       0   0   0   0   0   0   0   0   0   0   
                      P000054643       0   0   0   0   0   0   0   0   0   0   
                      P000054644       0   0   0   0   0   0   0   0   0   0   
                      P000054645       0   0   0   0   0   0   0   0   0   0   
                      P000054646       0   0   0   0   0   0   0   0   0   0   
                      P000054647       0   0   0   0   0   0   0   0   0   0   
                      P000054648       0   0   0   0   0   0   0   0   0   0   
                      P000054649       0   0   0   0   0   0   0   0   0   0   
                      P000054650       0   0   0   0   0   0   0   0   0   0   
                      P000054651       0   0   0   0   0   0   0   0   0   0   
                      P000054652      

In [67]:
summ = p.groupby(['year_term', 'stage']).sum()


In [65]:
summ.loc[('2014.Fall', )]

,01,02,03,04,05,06,07,08,09,10,...,44,45,46,47,48,49,50,51,52,53
stage,,,,,,,,,,,,,,,,,,,,,
Accepted,0,1,1,7,7,11,24,39,50,61,...,190,183,175,140,138,132,125,122,28,0
Applied,5,9,14,22,49,74,94,125,146,158,...,324,316,309,256,253,249,240,235,63,0
Deposited,0,0,0,0,0,0,0,0,0,0,...,29,24,24,21,20,17,16,13,3,0


In [74]:
(summ.loc[('2013.Fall', )].transpose()).to_csv('2013.Fall.csv')
(summ.loc[('2014.Fall', )].transpose()).to_csv('2014.Fall.csv')
(summ.loc[('2015.Fall', )].transpose()).to_csv('2015.Fall.csv')
(summ.loc[('2016.Fall', )].transpose()).to_csv('2016.Fall.csv')

In [77]:
writer = pd.ExcelWriter('admissions_historic_data.xlsx')
terms = ['2010.Fall', '2011.Fall', '2012.Fall', '2013.Fall', '2014.Fall', '2015.Fall', '2016.Fall', '2017.Fall', '2018.Fall', ]
for t in terms:
    (summ.loc[(t, )].transpose()).to_excel(writer, t)
writer.save()